---
title: "Cheatsheet: Pandas DataFrame"
description: |
  Refer to the jupyter notebook for rendered code.
author: "Chi Zhang"
date: "2025-02-12"
categories: [Python]
sidebar: false
code-block-bg: true
code-block-border-left: true
jupyter: python3
format: 
  html:
    toc: true
    toc-depth: 3
    code-fold: false
    code-tools: false
    code-annotations: select
---

# Data analysis workflow

::: {.callout-note}

When working with data that need to be transformed, create a copy so that the original data is not affected. 

```python
planet_sub = planets.copy()[planets['Mass']>15]
```

:::

### Initial summary

```python
df.head()
df.columns
df.shape
df.describe()
df.dtypes
df.isnull().sum() # check missing
```

Unique values, count frequency

```python
df['col'].unique()
df['col'].value_counts() # same as R table()
```


### Clean

```python
# rename column
df = df.rename(columns = {'old_name': 'new_name'})
```

Convert data type, rounding

```python
df['num_col'].round(2) # 2 digits
df['num_col'].astype(str) # change into string

```


Missing value handling

```python
df = df.dropna() # drop na
df['col'] = df['col'].fillna(value) # impute with known value
df['col'] = df['col'].fillna(df['col'].mean()) # mean imputation
```

### Subsetting, filtering

```python
df_sub = df[['col1', 'col2']]
df_filter = df[df['col1'] < 25]
```

Might have to reset index afterwards, `df_new.reset_index()`

### Transform

Create new variable based on another

```python
# solution 1
planet_sub['pre2005'] = 'yes'
planet_sub.loc[planet_sub.year > 2005, "pre2005"] = "no"
# solution 2
planet_sub['pre2005'] = np.where(planet_sub['year']<2005, 'yes', 'no')
```

Sort

```python
planet_sub.sort_values('Mass', ascending=False)
planet_sub.sort_values(by = ['orbital_period', 'Mass'], ascending=False)
```

Transform with `lambda`

```python
df.transform(lambda x: x-x.mean())
```


## Visualization (basics)

```python
import seaborn as sns
import matplotlib.pyplot as plt

# histogram
sns.histplot(df['numvar'])
plt.show()

# boxplot
sns.boxplot(df['numvar'])

# barplot (categorical, raw data)
sns.countplot(df['cate_var'])

# scatter (2 var)
sns.scatterplot(x = 'var1', y = 'var2', hue = 'var3', data = df)
```









# Details

## Create a dataframe

There are multiple ways to do it: 

* `pd.Series`
* dictionary
* 2-d array
* combining mixed types 

### Create a dataframe by **combining Series**

In [ ]:
import pandas as pd
import numpy as np

# define population
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)

# define area
area_dict = {'California': 423967,
             'Texas': 695662,
             'New York': 141297,
             'Florida': 170312,
             'Illinois': 149995}
area = pd.Series(area_dict)

# now combine these two
states = pd.DataFrame({
    'population': population,
    'area': area
})
states

It is convenient to create a new column based on other columns, 

```python
states['density'] = states['population'] / states['area']
```

### Create a df from dictionary

In [ ]:
pd.DataFrame([{'a': 1, 'b': 2}, {'b': 3, 'c': 4}])

### Create a df from a 2-d array

In [ ]:
pd.DataFrame(np.random.rand(3, 2),
             columns=['foo', 'bar'],
             index=['a', 'b', 'c'])

### Mix types combination

This is equivalent to R `cbind`. Assuming that we have the following **np.array**: `X` (n by 2), `y` (n by 1), `yhat` (n by 1). We want to combine them together for easier visualization. 

* use `np.column.stack(())` first, then convert into pandas dataframe; This is similar to creating a numeric matrix, then convert it to dataframe in R.
* or, create a dataframe based on one of the variables first; then attach a second variable using `df['new_var'] = var2`. This is similar to `df$new_var <- var2` in R.

```python
# option 1
mat = np.column_stack((X, y, yhat)) # double bracket
df1 = pd.DataFrame(mat, columns = ['x1', 'x2', 'y', 'yhat'])

# option 2
df2 = pd.DataFrame(X, columns = ['x1', 'x2'])
df2['y'] = y
df2['yhat'] = yhat
```

## Create new columns

After the above steps (in previous section), we want to also create an indicator based on two of the variables. Can use `np.where` function.

```python
df['mismatch'] = np.where(df['y'] != df['yhat'], 1, 0)
```



## Selection

* rownames: index
* colnames: column
* select columns

```python
states.index # rownames (index)
states.columns # colnames
states['area'] # select column by name
```


Subsetting with `loc, iloc`

```python
# implicit index
states.iloc[:3, :2]

# explicit index
states.iloc[:'Illinois', :'pop']
```

Combine filtering

```python
states.loc[data.density > 100, ['pop', 'density']]
```
